In [1]:
import logging
logging.basicConfig(filename='cipi.log', format='%(message)s', level=logging.INFO)

from listens import *
import keras
from preprocess import *
from chat_iface import *
from chat_proc import *


C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['affirmasi', 'awake', 'broadcast', 'browser', 'cctv_lewatmana', 'direktori', 'goodbye', 'google', 'homeauto', 'jam', 'lihat_gambar', 'negasi', 'quran', 'salam', 'tokopedia', 'translate', 'wikipedia', 'youtube']
from data.affirmasi import affirmasi
from data.awake import awake
from data.broadcast import broadcast
from data.browser import browser
from data.cctv_lewatmana import cctv_lewatmana
from data.direktori import direktori
from data.goodbye import goodbye
from data.google import google
from data.homeauto import homeauto
from data.jam import jam
from data.lihat_gambar import lihat_gambar
from data.negasi import negasi
from data.quran import quran
from data.salam import salam
from data.tokopedia import tokopedia
from data.translate import translate
from data.wikipedia import wikipedia
from data.youtube import youtube


In [2]:
def process():
    ear.stop()
    pass
    print("Ini prosesnya")

In [3]:
def transcrib():
    global speech
    speech = transcribe_speech(s_iface)
    if speech == None:
        speech = ""
    print("Ini Transkrip:",speech)

In [4]:
ear = Listen()
ear.set_process(process)
#ear.start(thres=800, process=process)

In [5]:
model = keras.models.load_model("./model/chippy_v1.model")

In [6]:
from selenium import webdriver

headless = False
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--use-fake-ui-for-media-stream")
if headless:
    chrome_options.add_argument("--headless")  
    
def start_iface():
    s_iface = webdriver.Chrome(chrome_options=chrome_options)
    s_iface.get('https://translate.google.com/?#id/id')
    assert "Google Translate" in s_iface.title
    talk("saya siap",s_iface)
    return s_iface
    
def mainbrowser(url):
    global m_iface
    m_iface = webdriver.Chrome(chrome_options=chrome_options)
    m_iface.get('https://translate.google.com/?#id/id')

In [7]:
def relisten_hotword():
    global att_counter, cipi_recognized, attention, speech
    att_counter = 0
    cipi_recognized = False
    attention = False
    speech=""
    time.sleep(1)

In [8]:
def is_not_a_followup():
    if botstate['followup']=="None":
        return True
    return False    

In [ ]:
s_iface = start_iface()

In [ ]:
debug=True
cipi_recognized=False
ear.debug = False
feature_dim_1 = 20
feature_dim_2 = 11
channel = 1
awake = True

while not cipi_recognized:
    print("Start waiting for hotword...")
    ear.start(thres=1200, timeout=0)
    #ear.write_to_file('test.wav')
    dataaudio=np.array(ear.audiodata).flatten()
    sample = au2mfcc(dataaudio)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    y_pred = model.predict(sample_reshaped)
    y_max = np.max(y_pred) #nilai akurasi prediksi
    ypred = np.argmax(y_pred) #yg di prediksi adalah cipi
    cipi_recognized = (y_max>0.9 and ypred==1)
    if cipi_recognized: 
        
        #RANDOMIZE this
        playdetected()
        
        print("Recognized - Predicted:" ,get_labels()[0][ypred],y_max, ypred)

        attention=True
        att_counter = 0
        while attention:
            print("Attention is:",attention)
            rec_speech(s_iface)
            ear.debug = False
            ear.start(thres=800, timeout=5, process=transcrib)
            if speech: #ada kalimat yg disebut dalam bentuk string
                #RANDOMIZE THIS
                playdetected()
                reply,humanstate,botstate = sequence(speech,humanstate,botstate)
                if awake:
                    talk_until_complete(reply,s_iface)
                #if the reply not a followup
                if botstate['followup']=="None": 
                    print("not a followup")
                    relisten_hotword()
            else:
                print("blank speech detected")
                speech = " " #kasih sedikit spasi hack
                att_counter += 1
                if att_counter > 1:
                    relisten_hotword()
                    print("\n=============\nGo to wait mode")

Start waiting for hotword...
Ini prosesnya
Start waiting for hotword...
Ini prosesnya
Start waiting for hotword...
Ini prosesnya
Start waiting for hotword...
Ini prosesnya
Recognized - Predicted: cipi 0.94215786 1
Attention is: True
Ini Transkrip: saya siap

SENTENCE: saya siap

BEGIN
HUMANSTATE: {'sequence': 0, 'intent': 'homeauto', 'intentscore': 0.0014392026, 'followup': 'None', 'intentfu': 'None', 'context': 'None', 'content': 'None', 'lastmsg': 'saya siap'}
BOTSTATE: {'sequence': 0, 'intent': 'None', 'intentscore': 0.0, 'followup': 'None', 'intentfu': 'None', 'context': 'None', 'content': 'None', 'lastmsg': 'None', 'prompt': 'None', 'process': 'None'}
Sector C
Sector E
homeauto.input('saya siap')

END STATE
HUMAN STATE: {'sequence': 0, 'intent': 'homeauto', 'intentscore': 0.0014392026, 'followup': 'None', 'intentfu': 'None', 'context': 'None', 'content': 'None', 'lastmsg': 'saya siap'}
END BOTSTATE: {'sequence': 0, 'intent': 'homeauto', 'intentscore': 0.0, 'followup': 'ruang', 'in

BOTSTATE: {'sequence': 0, 'intent': 'None', 'intentscore': 0.0, 'followup': 'None', 'intentfu': 'homeauto', 'context': 'None', 'content': 'None', 'lastmsg': 'oke', 'prompt': 'oke', 'process': 'None'}
Sector C
Sector E
homeauto.input('nyalakan lampu')

END STATE
HUMAN STATE: {'sequence': 0, 'intent': 'homeauto', 'intentscore': 0.499877, 'followup': 'None', 'intentfu': 'None', 'context': 'None', 'content': 'None', 'lastmsg': 'nyalakan lampu'}
END BOTSTATE: {'sequence': 0, 'intent': 'None', 'intentscore': 0.0, 'followup': 'None', 'intentfu': 'homeauto', 'context': 'None', 'content': 'None', 'lastmsg': 'oke', 'prompt': 'oke', 'process': 'None'}

REPLY: oke
X
not a followup
Start waiting for hotword...
Ini Transkrip: 
Recognized - Predicted: cipi 0.9999894 1
Attention is: True
Ini Transkrip: nyalakan lampu ruangan tengah

SENTENCE: nyalakan lampu ruangan tengah

BEGIN
HUMANSTATE: {'sequence': 0, 'intent': 'homeauto', 'intentscore': 0.0025209482, 'followup': 'None', 'intentfu': 'None', 'cont

Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Recognized - Predicted: cipi 0.9888341 1
Attention is: True
Ini Transkrip: 
blank speech detected
Attention is: True
Ini Transkrip: Berapa hasil

SENTENCE: Berapa hasil

BEGIN
HUMANSTATE: {'sequence': 0, 'intent': 'tokopedia', 'intentscore': 0.04788972, 'followup': 'None', 'intentfu': 'None', 'context': 'None', 'content': 'None', 'lastmsg': 'Berapa hasil'}
BOTSTATE: {'sequence': 0, 'intent': 'chitchat', 'intents

Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transkrip: 
Start waiting for hotword...
Ini Transk